## Grayscale images

Note: This chapter deals with grayscale images only (for now)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.signal
import skimage.io
import skimage.color
import skimage.exposure
from ipywidgets import interact

#I = skimage.data.astronaut()
I = skimage.io.imread('img/Trees.jpg')
plt.imshow(I)

In [ ]:
I = skimage.color.rgb2gray(I)
plt.imshow(I, cmap='gray')

## Image histogram

Histogram = the number of occurrences of every pixel value

In [ ]:
_,_,_=plt.hist(I.ravel(), bins=256)

## Histogram equalization

Histogram equalization:

- arrange all pixels in order, from smallest to largest 
- set pixel value proportional to position of pixel in list
  - darkest pixel becomes 0
  - whitest pixed becomes 255
  - the median pixel becomes 127
  - ... 

Advantage: 

- Increase contrast

In [ ]:
plt.imshow(I, cmap='gray')

In [ ]:
Iheq = skimage.exposure.equalize_hist(I)
plt.imshow(Iheq, cmap='gray')

Original histogram: 

In [ ]:
_,_,_=plt.hist(I.ravel(), bins=256)

Final histogram:

In [ ]:
_,_,_=plt.hist(Iheq.ravel(), bins=256)

## Adaptive (local) histogram equalization

- Apply histogram equalization based on a **local window** around current position

Advantage:

- images with unequal illumination

In [ ]:
P = skimage.data.page()
plt.imshow(P, cmap='gray')

In [ ]:
Pheq = skimage.exposure.equalize_hist(P)
plt.imshow(Pheq, cmap='gray')

In [ ]:
Pheqadapt = skimage.exposure.equalize_adapthist(P)
plt.imshow(Pheqadapt, cmap='gray')

In [ ]:
_,_,_=plt.hist(Pheqadapt.ravel(), bins=256)

## Thresholding

Thresholding = Make an image binary, by comparing with a threshold T

Example: Threshold $T = 128$

In [ ]:
plt.imshow(P > 128, cmap='gray')

Example: Threshold $T = 165$

In [ ]:
plt.imshow(P > 165, cmap='gray')

In [ ]:
@interact(T=(0,255,1))
def thresh(T):
  plt.imshow(P > T, cmap='gray')

Thresholding the image after adaptive equalization of histogram:

In [ ]:
plt.imshow(Pheqadapt > 0.30, cmap='gray')

In [ ]:
@interact(T=(0,1,0.01))
def thresh(T):
  plt.imshow(Pheqadapt > T, cmap='gray')

## Automatic thresholding with Otsu's method

- Find threshold such as to **minimize intra-class variance**
  - equivalently, maximize inter-class variance

![Otsu's method explanation](img/Otsu.jpg)

In [ ]:
from skimage.filters import threshold_otsu

T_Otsu = threshold_otsu(Pheqadapt)
print(f"Threshold found by Otsu method = {T_Otsu}")
plt.imshow(Pheqadapt > T_Otsu, cmap='gray')

## Adaptive thresholding

Adaptove = Find threshold based only on the **surrounding region** of a pixel
  
- Can use any method: mean value of region, Otsu, etc

- `threshold_local()` from `skimage` package: specify region size, and an offset to subtract from region mean


In [ ]:
from skimage.filters import threshold_local

def thresh_local(block_size=25, offset=0.1):
  Tadapt = threshold_local(Pheqadapt, block_size=block_size, offset=offset)
  plt.imshow(Pheqadapt > Tadapt, cmap='gray')
  return np.array(Pheqadapt > Tadapt, dtype=int)

Pthr = thresh_local()

skimage.io.imsave('img/PageBinary.png', Pthr, check_contrast=False)

Interactive:

In [ ]:
from skimage.filters import threshold_local

@interact(block_size=(3, 128, 2), offset=(-0.2, 0.2, 0.01))
def thresh_local(block_size, offset):
  Tadapt = threshold_local(Pheqadapt, block_size=block_size, offset=offset)
  plt.imshow(Pheqadapt > Tadapt, cmap='gray')